<a href="https://colab.research.google.com/github/malgorzatagwinner/text_classification/blob/main/GloveTsai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy

!pip install umap-learn
numpy.version.version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 9.5 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=1bb928c9ce7fd155b295fa37ae97faecd6ea47ee047d4ad111d22ec2aa28c2c9
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=84631e9b408a601130fd1bf86204a06f5c53c5e5b405bee57bc7061b18af227c
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent


'1.21.6'

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy
import umap
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
import pandas as pd
from sklearn.datasets import load_digits
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = open("/content/drive/My Drive//vectors.txt", "r")

words = []
numbers = []
for line in file:
    tmp = line.split()
    if len(tmp) != 51:
        tmp.insert(0, ' ')
    words.append(tmp.pop(0))
    for i in range(len(tmp)):
        if tmp[i] == '.':
            tmp[i] = 0
        else:
            tmp[i] = float(tmp[i])
    numbers.append(tmp)
#    print(len(numbers), " ", len(tmp), " ", words[-1])
file.close()
print(len(words))
# numbers = numpy.array(numbers)

words_df = pd.DataFrame(numbers, index=words)
words_df = words_df[~words_df.index.duplicated(keep='first')]
words_df
# word_index = {w: i for (i, w) in enumerate(words)}
print(words_df.describe())
# numbers_f = []
# for i in numbers:
#     for j in range(len(word_index)):
#         if word_index[j] == i:
#             numbers_f = i

#numbers = numpy.array(numbers_f)

45408
                 0             1             2             3             4   \
count  14022.000000  14022.000000  14022.000000  14022.000000  14022.000000   
mean      -0.019757     -0.002807     -0.019000      0.020436      0.166648   
std        0.456095      0.466246      0.476108      0.450243      0.404736   
min       -1.851942     -2.412055     -2.074380     -1.471966     -1.556067   
25%       -0.323677     -0.322051     -0.324895     -0.291098     -0.096718   
50%       -0.009887     -0.013641     -0.013508     -0.013055      0.172664   
75%        0.288384      0.308448      0.300339      0.299339      0.431783   
max        1.604943      2.251804      1.754476      2.005872      2.426469   

                 5             6             7             8             9   \
count  14022.000000  14022.000000  14022.000000  14022.000000  14022.000000   
mean      -0.189493     -0.010187     -0.016445      0.066486      0.033756   
std        0.425350      0.459400      0.4796

In [ ]:
from tqdm.notebook import tqdm_notebook
import string
import glob
import pickle

In [ ]:
import numpy as np
from math import log
from sklearn.linear_model import LinearRegression
import os
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
glove_sum = []
PATH = '/content/drive/My Drive/Colab Notebooks/glove/'
from os.path import exists
authors_sl = []
authors_wl = []
words_l = []

authors_d = {}
embeddings_l = []
for filename in (tqdm_notebook(sorted(glob.glob(f'/content/drive/My Drive/corpus/*.txt')))):
    filen = PATH + filename.rpartition('/')[2] + '.pickle'
    if False and exists(filen): # load
        with open(filen, 'rb') as f:
            zdania = pickle.load(f)
            print(zdania)
    else: # calculate and save
        sents = []
        with open(filename, 'r', encoding="utf8", errors='ignore') as in_file: 
             text = in_file.read().replace('\n', ' ')
        sents = nltk.word_tokenize(text)
        text = None
        _, tail = os.path.split(filename)
        auth = tail.split("_")[0]
        if not auth in authors_d:
            authors_d[auth] = len(authors_d)

        for i in sents:
            try:
                words_l.append(words_df.loc[i])
                authors_wl.append(authors_d[auth])
            except:
                continue
        authors_sl.append(authors_d[auth])
        sents = None
        

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
##### przerabiana wersja

from nltk.tokenize import word_tokenize
glove_sum = []
glove_docs = []
PATH = '/content/drive/My Drive/Colab Notebooks/glove_tsai/'
from os.path import exists
authors_sl = []
words_l = []

authors_d = {}
embeddings_l = []
for filename in (tqdm_notebook(sorted(glob.glob(f'/content/drive/My Drive/corpus/*.txt')))):
    filen = PATH + filename.rpartition('/')[2] + '.pickle'
    if False and exists(filen): # load
        with open(filen, 'rb') as f:
            zdania = pickle.load(f)
            print(zdania)
    else: # calculate and save
        sents = []
        with open(filename, 'r', encoding="utf8", errors='ignore') as in_file: 
             text = in_file.read().replace('\n', ' ')
        sents = nltk.sent_tokenize(text)
        text = None
        _, tail = os.path.split(filename)
        auth = tail.split("_")[0]
        if not auth in authors_d:
            authors_d[auth] = len(authors_d)

        for sent in sents:
            for w in sent:
                try:
                    words_l.append(words_df.loc[w])
                except:
                    continue
            glove_docs.append(words_l)
        authors_sl.append(authors_d[auth])
        with open(filen, 'wb') as f:
            pickle.dump(glove_docs, f, pickle.HIGHEST_PROTOCOL)
        glove_sum.append(glove_docs)
        sents = None
        

Wszystkie słowa ze wszystkich dokumentów, nie dzieląc ich na dokumenty (bo za mało ramu i słowa się powtarzają)

In [ ]:
len(words_l)

In [ ]:
batches = []
authors_b = []

for j in range(0, len(glove_sum[i])-200, 200):
    batches.append(glove_sum[i][j:j+200])
    authors_b.append(authors_sl[i])
    batches.append(s)

batches = np.asarray(batches)#.astype('float32')
authors_b = np.asarray(authors_b)

In [ ]:
len(authors_b)

In [ ]:
pip install tsai

In [ ]:
from tsai.all import *

In [ ]:
X, y, splits = get_classification_data('ECG200', split_data=False)
batch_tfms = TSStandardize()

splits = get_splits(authors_b, valid_size=.2, stratify=True, random_state=23, shuffle=True)
clf = TSClassifier(batches authors_b, splits=splits, path='models', arch=XceptionTime, batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph())
clf.fit_one_cycle(100, 3e-4)
clf.export("clf.pkl")